In [1]:
import pandas as pd
import os
import numpy as np

In [43]:
path = r"C:\Users\janse\OneDrive\Bureaublad\Master\Data Mining Techniques\Assignment 2"
testpath = os.path.join(path, 'test_set_VU_DM.csv')
trainpath = os.path.join(path, 'training_set_VU_DM.csv')
samplepath = os.path.join(path, 'train_sample.csv')
train = pd.read_csv(trainpath, nrows=2000000)
train.date_time = pd.to_datetime(train.date_time)


# Extract house features.
Price_usd is transformed logaritmically to improve distribution.
Nulls are filled with the average of the column.
4603 houses in the dataset.

In [ ]:
prop_cols = [x  for x in train.columns if x.startswith('prop_')]
prop_cols.append('price_usd')
avg_house_props = sample[prop_cols].groupby('prop_id').agg('mean')
avg_house_props.price_usd_log = np.log(avg_house_props.price_usd)
avg_house_props.fillna(avg_house_props.mean(), inplace=True)

In [ ]:
from sklearn.cluster import KMeans

class clusters:
    def __init__(self, clusters, searches):
        kmean = KMeans(n_clusters=clusters)
        kmean.fit(searches)
        self.clusters = kmean
    def assign(self, search_terms):
        return self.clusters.predict(search_terms)

cols = [x for x in train.columns if x.startswith('srch_')]
search_info = train[cols].drop('srch_query_affinity_score', axis=1).drop_duplicates()

search_info.drop(['srch_id','srch_destination_id'], axis=1, inplace=True)
groups = clusters(10, search_info)

search_info['search_group'] = groups.assign(search_info)
search_info.head(20)


# Popular properties

In [ ]:
# bookings = train[['prop_id', 'booking_bool']].groupby('prop_id').agg('sum').reset_index()
# bookings = bookings.sort_values('booking_bool',ascending=False).rename({'booking_bool':'bookings'},axis=1)
# days_occupied = train[train.booking_bool == 1][['prop_id', 'srch_length_of_stay']].groupby('prop_id').agg('sum').reset_index()
# days_occupied = days_occupied.rename({'srch_length_of_stay':'days_booked'},axis=1)
# popular_props = bookings.merge(days_occupied, on='prop_id', how='outer').sort_values('days_booked', ascending=False)

popular_props = train[['prop_id','booking_bool', 'click_bool']].groupby('prop_id').agg('mean').sort_values('booking_bool', ascending = False)
#train[['prop_id','search_id']].groupby('prop_id').agg('count').reset_index().rename({'search_id': 'popups'}, axis=1)
popular_props = popular_props.merge(train[['prop_id','srch_id']].groupby('prop_id').agg('count').reset_index().rename({'srch_id': 'popups'}, axis=1), on='prop_id')
popular_props = popular_props[popular_props.popups > 5].head()

# Benchmark model
Use the most popular houses (based on the amount of bookings) to rank houses from the search term.

In [ ]:
result = train[['prop_id','srch_id']].merge(popular_props[popular_props.popups > 5][['prop_id', 'booking_bool']][popular_props.popups > 5], on='prop_id', how='outer')
result = result.fillna(0).sort_values(['srch_id', 'booking_bool'], ascending=[True, False])
labels = train[['srch_id','prop_id', 'click_bool', 'booking_bool']]
labels['label'] = labels['click_bool'] + 4 * labels['booking_bool']
result = result.merge(labels[['prop_id', 'srch_id', 'label']], on=['prop_id', 'srch_id'])


# Features

In [49]:
# Cumulative competitor scores

cols = [col for col in train.columns if col.endswith('rate_percent_diff')]
cols2 = [col for col in train.columns if col.endswith('_rate')]
cols3 = [col for col in train.columns if col.endswith('_inv')]

train2 = train
train2['comp_rate_percent_diff_cumm'] = 0

for col1, col2, col3 in zip(cols, cols2, cols3):
    train2['comp_rate_percent_diff_cumm'] += train2[col1].fillna(0) * train2[col2].fillna(0) * (1 - train2[col3].fillna(0))

train2 = train2.drop(cols, axis=1)
train2 = train2.drop(cols2, axis=1)
train2 = train2.drop(cols3, axis=1)

0      0.0
1      0.0
2      0.0
3    -15.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10    11.0
11    26.0
12     0.0
13     0.0
14     0.0
15    28.0
16     5.0
17     0.0
18     0.0
19     0.0
Name: comp_rate_percent_diff_cumm, dtype: float64
['comp1_rate_percent_diff', 'comp2_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate_percent_diff', 'comp5_rate_percent_diff', 'comp6_rate_percent_diff', 'comp7_rate_percent_diff', 'comp8_rate_percent_diff']
Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
      

In [ ]:
from sklearn.cluster import KMeans

class search_clusters:
    def __init__(self, clusters, searches):
        kmean = KMeans(n_clusters=clusters)
        kmean.fit(searches)
        self.clusters = kmean
    def assign(self, search_terms):
        return self.clusters.predict(search_terms)

cols = [x for x in train2.columns if x.startswith('srch_')]
clusters = train2[cols].drop('srch_query_affinity_score', axis=1).drop_duplicates()

clusters.drop(['srch_id','srch_destination_id'], axis=1, inplace=True)
groups = search_clusters(30, clusters)

clusters['search_group'] = groups.assign(clusters)
clusters.head(20)

# Evaluation

In [18]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [21]:
def eval_ndcg(df, srch_id_col, predict_col, value_col):
    df = df.sort_values(predict_col)
    k = 5
    ndcgs = []
    for i, srchid in enumerate(df[srch_id_col].unique()):
        if i % 10000 == 0 and i != 0:
            print(i)
            print(np.mean(ndcgs))
            print(r)
        r = df[df[srch_id_col] == srchid][value_col]
        ndcgs.append(ndcg_at_k(r,k))

    print(np.mean(ndcgs))


In [52]:
cols = [x for x in train2.columns if x.startswith('srch_')]
cols.extend([x for x in train2.columns if x.startswith('prop_')])
cols.extend(['price_usd','orig_destination_distance', 'promotion_flag'])
print(cols)

X = train2.fillna(0)
y = train2.click_bool + 4 * train2.booking_bool
X = X.drop(['click_bool','booking_bool', 'date_time'], axis=1)

['srch_id', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'srch_query_affinity_score', 'prop_country_id', 'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'prop_log_historical_price', 'price_usd', 'orig_destination_distance', 'promotion_flag']


In [ ]:
from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  

from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train)  

y_pred = classifier.predict(X_test)

In [ ]:
results = pd.DataFrame({'srch_id': X_test.srch_id, 'predicted': y_pred, 'actual' : y_test})
eval_score = eval_ndcg(results, 'srch_id', 'predicted', 'actual')
